#### Setup

In [1]:
import pandas as pd
import numpy as np

import os
import glob

from PIL import Image

In [2]:
path = 'C:/Users/aks13/Dropbox (Partners HealthCare)/ML stuff/NSLT_CT_IMG_samp_normal/'
directory = os.listdir(path)

#### Image resizing and moving

In [3]:
# resize all images in the folder to standard size of 500 x 500
# new images have '_renamed' at the end of the og filename

def resize_img():
    for item in directory:
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            f, e = os.path.splitext(path+item)
            imResize = im.resize((500, 500), Image.ANTIALIAS) #this smooths the image edges, maybe dont do? not sure
            imResize.save(f + '_resized.png', 'PNG', quality = 90) #95 is recommended max, 75 is avg?

In [4]:
#### ONLY RUN THIS WHEN YOU NEED TO RANDOMIZE A NEW FOLDER
resize_img()

C:\Users\aks13\AppData\Local\Temp\ipykernel_1664\1604919891.py:9: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  imResize = im.resize((500, 500), Image.ANTIALIAS) #this smooths the image edges, maybe dont do? not sure


In [5]:
# move the resized images to a new folder
# NSLT_CT_IMG_samp_resize_V2 to train on "normal" images only
import shutil

for file in glob.glob('C:/Users/aks13/Dropbox (Partners HealthCare)/ML stuff/NSLT_CT_IMG_samp_normal/*resized.png'):
    shutil.move(file, 'C:/Users/aks13/Dropbox (Partners HealthCare)/ML stuff/NSLT_CT_IMG_samp_resize_V2/')

In [6]:
# new directory of resized images --> /NSLT_CT_IMG_samp_resize/

os.getcwd()

'C:\\Users\\aks13\\Dropbox (Partners HealthCare)\\ML stuff'

In [7]:
# new path
path = 'C:/Users/aks13/Dropbox (Partners HealthCare)/ML stuff/NSLT_CT_IMG_samp_resize_V2/'

In [8]:
# grab filenames and put into a list.

def get_imlist(path):
    return [os.path.join(path,f) for f in os.listdir(path) if f.endswith('.png')]

In [9]:
filepaths = get_imlist(path)
filepaths

['C:/Users/aks13/Dropbox (Partners HealthCare)/ML stuff/NSLT_CT_IMG_samp_resize_V2/100002_0_0OPAGELSPLUSD3602.512080.00.11.5_resized.png',
 'C:/Users/aks13/Dropbox (Partners HealthCare)/ML stuff/NSLT_CT_IMG_samp_resize_V2/100002_0_0OPAGELSPLUSLUNG3602.512080.00.11.5_resized.png',
 'C:/Users/aks13/Dropbox (Partners HealthCare)/ML stuff/NSLT_CT_IMG_samp_resize_V2/100002_1_1OPAGELSPLUSD3602.512080.00.11.5_resized.png',
 'C:/Users/aks13/Dropbox (Partners HealthCare)/ML stuff/NSLT_CT_IMG_samp_resize_V2/100002_1_1OPAGELSPLUSLUNG3602.512080.00.11.5_resized.png',
 'C:/Users/aks13/Dropbox (Partners HealthCare)/ML stuff/NSLT_CT_IMG_samp_resize_V2/100002_2_2OPAGELSPLUSD3602.512080.00.11.5_resized.png',
 'C:/Users/aks13/Dropbox (Partners HealthCare)/ML stuff/NSLT_CT_IMG_samp_resize_V2/100012_1_1OPASEVZOOMB30f2802.012070.040.0null_resized.png',
 'C:/Users/aks13/Dropbox (Partners HealthCare)/ML stuff/NSLT_CT_IMG_samp_resize_V2/100012_1_1OPASEVZOOMB50f2802.012070.040.0null_resized.png',
 'C:/Users/ak

In [10]:
filepaths_2 = [item.split('/') for item in filepaths]
filepaths_2

[['C:',
  'Users',
  'aks13',
  'Dropbox (Partners HealthCare)',
  'ML stuff',
  'NSLT_CT_IMG_samp_resize_V2',
  '100002_0_0OPAGELSPLUSD3602.512080.00.11.5_resized.png'],
 ['C:',
  'Users',
  'aks13',
  'Dropbox (Partners HealthCare)',
  'ML stuff',
  'NSLT_CT_IMG_samp_resize_V2',
  '100002_0_0OPAGELSPLUSLUNG3602.512080.00.11.5_resized.png'],
 ['C:',
  'Users',
  'aks13',
  'Dropbox (Partners HealthCare)',
  'ML stuff',
  'NSLT_CT_IMG_samp_resize_V2',
  '100002_1_1OPAGELSPLUSD3602.512080.00.11.5_resized.png'],
 ['C:',
  'Users',
  'aks13',
  'Dropbox (Partners HealthCare)',
  'ML stuff',
  'NSLT_CT_IMG_samp_resize_V2',
  '100002_1_1OPAGELSPLUSLUNG3602.512080.00.11.5_resized.png'],
 ['C:',
  'Users',
  'aks13',
  'Dropbox (Partners HealthCare)',
  'ML stuff',
  'NSLT_CT_IMG_samp_resize_V2',
  '100002_2_2OPAGELSPLUSD3602.512080.00.11.5_resized.png'],
 ['C:',
  'Users',
  'aks13',
  'Dropbox (Partners HealthCare)',
  'ML stuff',
  'NSLT_CT_IMG_samp_resize_V2',
  '100012_1_1OPASEVZOOMB30f2

In [11]:
#keep only the filename
filepaths_3 = [item.pop() for item in filepaths_2]
filepaths_3

# rename
filenames_imgs = filepaths_3

#### Converting resized images into arrays

In [12]:
images = []

# make empty array --> 500x500 size
np_imgs = np.zeros((500, 500))

In [13]:
# convert each image into an array
count = 0

for f in glob.iglob('C:/Users/aks13/Dropbox (Partners HealthCare)/ML stuff/NSLT_CT_IMG_samp_resize_V2/*.png'):
    images = Image.open(f).convert('L') #convert to b/w THEN flatten, L is grayscale
    images = np.asarray(images) # create the arrays...
    images = [item.flatten() for item in images] #...then flatten before adding
    np_imgs[count,:] #flattened image vector
    count = count + 1
    
images = np.array(images)

# add back filenames to this and make a new list
# 1:1 ratio filename:array once flattened to 1D

In [14]:
len(images)

500

In [15]:
images

array([[58, 58, 58, ..., 36, 34, 34],
       [58, 58, 58, ..., 36, 34, 34],
       [58, 58, 59, ..., 35, 34, 34],
       ...,
       [75, 78, 80, ..., 86, 84, 81],
       [75, 77, 80, ..., 86, 83, 81],
       [75, 77, 79, ..., 85, 82, 81]], dtype=uint8)

In [16]:
len(filenames_imgs)
# these two lists have to match in length or they wont combine correctly

500

#### Convert arrays into a dataframe, matching filenames as the index

In [17]:
# try --> just have the df from the flat_imgs arrays first
# then insert the filenames as the first column

df_v1 = pd.DataFrame(images, index=filenames_imgs)
#df_v1 = pd.DataFrame(images[0:10], index=filenames_imgs[0:10]) #test 10 output

df_v1

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
100002_0_0OPAGELSPLUSD3602.512080.00.11.5_resized.png,58,58,58,59,59,59,59,61,63,64,...,33,33,34,35,37,37,37,36,34,34
100002_0_0OPAGELSPLUSLUNG3602.512080.00.11.5_resized.png,58,58,58,59,59,59,59,61,63,64,...,33,33,34,35,37,37,37,36,34,34
100002_1_1OPAGELSPLUSD3602.512080.00.11.5_resized.png,58,58,59,59,59,59,59,61,63,64,...,33,34,34,35,36,37,36,35,34,34
100002_1_1OPAGELSPLUSLUNG3602.512080.00.11.5_resized.png,59,60,60,60,60,60,60,62,63,64,...,34,34,34,36,36,36,36,35,34,34
100002_2_2OPAGELSPLUSD3602.512080.00.11.5_resized.png,59,60,60,60,60,60,60,62,63,64,...,34,34,34,36,36,36,35,34,34,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216093_2_2OPAGELSPR16BONE2921.21204834.91.4_resized.png,76,78,81,83,85,86,89,91,93,95,...,101,97,96,94,92,91,89,88,84,82
216093_2_2OPAGELSPR16BONE2922.51204834.91.4_resized.png,76,78,80,82,84,86,88,90,92,94,...,101,97,96,94,92,91,89,88,85,82
216093_2_2OPAGELSPR16STANDARD2851.21204834.91.4_resized.png,75,78,80,82,84,86,87,89,92,94,...,100,96,95,94,92,90,88,86,84,81
218260_0_0OPASEVZOOMB50f350212080402_resized.png,75,77,80,82,84,86,87,89,92,94,...,100,95,95,93,92,90,88,86,83,81


#### PCA

In [18]:
from sklearn.decomposition import PCA
#from sklearn.preprocessing import StandardScaler #dont scale outliers lol

X = df_v1.values
X.shape

(500, 500)

In [19]:
X

array([[58, 58, 58, ..., 36, 34, 34],
       [58, 58, 58, ..., 36, 34, 34],
       [58, 58, 59, ..., 35, 34, 34],
       ...,
       [75, 78, 80, ..., 86, 84, 81],
       [75, 77, 80, ..., 86, 83, 81],
       [75, 77, 79, ..., 85, 82, 81]], dtype=uint8)

In [ ]:
# old scaled method
# trying without

# scale the data
##scaler = StandardScaler()
##scaler.fit(X)
##X_scaled = scaler.transform(X)
##X_scaled

In [20]:
pca_v1 = PCA(n_components=5, random_state=42)
pca_v1.fit(X)
#pca_v1.fit(X_scaled)

PCA(n_components=5, random_state=42)

In [21]:
X_pca_v1 = pca_v1.transform(X)
#X_pca_v1 = pca_v1.transform(X_scaled)
X_pca_v1.shape

(500, 5)

In [22]:
X #original df from pca arrays
df_v1 # before doing anything
X_pca_v1 #array of PCA values that were fit/transformed --> use this in IF

array([[ -48.41611081, -402.69352749,  493.95919708, -161.26134505,
         176.21959255],
       [ -48.1618208 , -402.63047302,  494.02757088, -161.33443259,
         176.29430493],
       [ -37.78593545, -405.76873695,  493.54565218, -163.73318569,
         183.88022185],
       ...,
       [1590.94094386,  245.38386573, -213.07127217,   -9.98368316,
         -12.38785757],
       [1591.47658765,  247.92630344, -209.69499804,   -9.19832021,
          -7.02835336],
       [1592.89386413,  250.72765109, -207.10645572,   -9.32884592,
          -3.18802352]])

In [23]:
df_v1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 100002_0_0OPAGELSPLUSD3602.512080.00.11.5_resized.png to 218260_1_1OPASEVZOOMB30f346212080402_resized.png
Columns: 500 entries, 0 to 499
dtypes: uint8(500)
memory usage: 248.0+ KB


#### Isolation Forest

In [24]:
from sklearn.ensemble import IsolationForest
# put the PCA array back to a DF to preserve the new components

pca_v1_df = pd.DataFrame(X_pca_v1, index=filenames_imgs)
pca_v1_df.columns = ['PCA 1', 'PCA 2', 'PCA 3', 'PCA 4', 'PCA 5']
pca_v1_df

,PCA 1,PCA 2,PCA 3,PCA 4,PCA 5
100002_0_0OPAGELSPLUSD3602.512080.00.11.5_resized.png,-48.416111,-402.693527,493.959197,-161.261345,176.219593
100002_0_0OPAGELSPLUSLUNG3602.512080.00.11.5_resized.png,-48.161821,-402.630473,494.027571,-161.334433,176.294305
100002_1_1OPAGELSPLUSD3602.512080.00.11.5_resized.png,-37.785935,-405.768737,493.545652,-163.733186,183.880222
100002_1_1OPAGELSPLUSLUNG3602.512080.00.11.5_resized.png,-19.435297,-410.697330,487.795051,-168.911256,194.991631
100002_2_2OPAGELSPLUSD3602.512080.00.11.5_resized.png,-5.050823,-414.881779,485.411161,-172.827081,205.200044
...,...,...,...,...,...
216093_2_2OPAGELSPR16BONE2921.21204834.91.4_resized.png,1585.078076,241.033359,-223.337915,-9.659986,-25.266140
216093_2_2OPAGELSPR16BONE2922.51204834.91.4_resized.png,1589.774764,242.955436,-218.908422,-10.354376,-18.588412
216093_2_2OPAGELSPR16STANDARD2851.21204834.91.4_resized.png,1590.940944,245.383866,-213.071272,-9.983683,-12.387858
218260_0_0OPASEVZOOMB50f350212080402_resized.png,1591.476588,247.926303,-209.694998,-9.198320,-7.028353


In [25]:
anomaly_pca = ['PCA 1', 'PCA 2', 'PCA 3', 'PCA 4', 'PCA 5']

if_model = IsolationForest(max_samples=100, contamination=0.1, random_state=42)
if_model.fit(pca_v1_df[anomaly_pca]) # use the PCA'd df

C:\Users\aks13\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


IsolationForest(contamination=0.1, max_samples=100, random_state=42)

In [26]:
pca_v1_df['anomaly score'] = if_model.decision_function(pca_v1_df[anomaly_pca])
pca_v1_df['anomaly'] = if_model.predict(pca_v1_df[anomaly_pca])
pca_v1_df

,PCA 1,PCA 2,PCA 3,PCA 4,PCA 5,anomaly score,anomaly
100002_0_0OPAGELSPLUSD3602.512080.00.11.5_resized.png,-48.416111,-402.693527,493.959197,-161.261345,176.219593,0.022016,1
100002_0_0OPAGELSPLUSLUNG3602.512080.00.11.5_resized.png,-48.161821,-402.630473,494.027571,-161.334433,176.294305,0.022016,1
100002_1_1OPAGELSPLUSD3602.512080.00.11.5_resized.png,-37.785935,-405.768737,493.545652,-163.733186,183.880222,0.021247,1
100002_1_1OPAGELSPLUSLUNG3602.512080.00.11.5_resized.png,-19.435297,-410.697330,487.795051,-168.911256,194.991631,0.023212,1
100002_2_2OPAGELSPLUSD3602.512080.00.11.5_resized.png,-5.050823,-414.881779,485.411161,-172.827081,205.200044,0.022323,1
...,...,...,...,...,...,...,...
216093_2_2OPAGELSPR16BONE2921.21204834.91.4_resized.png,1585.078076,241.033359,-223.337915,-9.659986,-25.266140,0.094787,1
216093_2_2OPAGELSPR16BONE2922.51204834.91.4_resized.png,1589.774764,242.955436,-218.908422,-10.354376,-18.588412,0.096733,1
216093_2_2OPAGELSPR16STANDARD2851.21204834.91.4_resized.png,1590.940944,245.383866,-213.071272,-9.983683,-12.387858,0.092685,1
218260_0_0OPASEVZOOMB50f350212080402_resized.png,1591.476588,247.926303,-209.694998,-9.198320,-7.028353,0.090220,1


In [27]:
pca_v1_df # -1 = outlier (anomaly), 1 = non-outlier (normal)

,PCA 1,PCA 2,PCA 3,PCA 4,PCA 5,anomaly score,anomaly
100002_0_0OPAGELSPLUSD3602.512080.00.11.5_resized.png,-48.416111,-402.693527,493.959197,-161.261345,176.219593,0.022016,1
100002_0_0OPAGELSPLUSLUNG3602.512080.00.11.5_resized.png,-48.161821,-402.630473,494.027571,-161.334433,176.294305,0.022016,1
100002_1_1OPAGELSPLUSD3602.512080.00.11.5_resized.png,-37.785935,-405.768737,493.545652,-163.733186,183.880222,0.021247,1
100002_1_1OPAGELSPLUSLUNG3602.512080.00.11.5_resized.png,-19.435297,-410.697330,487.795051,-168.911256,194.991631,0.023212,1
100002_2_2OPAGELSPLUSD3602.512080.00.11.5_resized.png,-5.050823,-414.881779,485.411161,-172.827081,205.200044,0.022323,1
...,...,...,...,...,...,...,...
216093_2_2OPAGELSPR16BONE2921.21204834.91.4_resized.png,1585.078076,241.033359,-223.337915,-9.659986,-25.266140,0.094787,1
216093_2_2OPAGELSPR16BONE2922.51204834.91.4_resized.png,1589.774764,242.955436,-218.908422,-10.354376,-18.588412,0.096733,1
216093_2_2OPAGELSPR16STANDARD2851.21204834.91.4_resized.png,1590.940944,245.383866,-213.071272,-9.983683,-12.387858,0.092685,1
218260_0_0OPASEVZOOMB50f350212080402_resized.png,1591.476588,247.926303,-209.694998,-9.198320,-7.028353,0.090220,1


In [29]:
pca_v1_df.to_csv('nonscaled-check_01042023.csv')